<a href="https://colab.research.google.com/github/NISHAKAR06/En-ta_model/blob/main/Copy_of_eng_tam_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Model-1**

In [ ]:
!pip install datasets

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import json
import torch

# Load and prepare dataset
with open("/content/en_ta_instruct.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)

data = [{"english": d["input"], "thanglish": d["output"]} for d in raw_data if d.get("input") and d.get("output")]
dataset = Dataset.from_list(data).train_test_split(test_size=0.1)

# Load tokenizer and model
model_name = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name, legacy=False)  # Set legacy=False to avoid legacy behavior
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Preprocessing function
def preprocess(example):
    input_text = "translate English to Thanglish: " + example["english"]
    target_text = example["thanglish"]
    inputs = tokenizer(input_text, padding="max_length", truncation=True, max_length=128)
    targets = tokenizer(target_text, padding="max_length", truncation=True, max_length=128)
    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized_dataset = dataset.map(preprocess, remove_columns=["english", "thanglish"])

# Training arguments (without evaluation_strategy)
training_args = TrainingArguments(
    output_dir="./t5-thanglish",
    learning_rate=3e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    fp16=torch.cuda.is_available(),
    # Removed evaluation_strategy
)

# Trainer initialization
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Save final model
model.save_pretrained("t5-thanglish-final")
tokenizer.save_pretrained("t5-thanglish-final")


Map:   0%|          | 0/14375 [00:00<?, ? examples/s]

Map:   0%|          | 0/1598 [00:00<?, ? examples/s]

<ipython-input-5-45d258c37814>:44: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,0.236200
1000,0.091900


**Accuracy**

In [ ]:
from datasets import load_metric
from datasets import load_dataset, Dataset
import json
import torch
import numpy as np  # Import numpy


metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

**testing**

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load model & tokenizer
model = T5ForConditionalGeneration.from_pretrained("t5-thanglish-final")
tokenizer = T5Tokenizer.from_pretrained("t5-thanglish-final")

# Translation function
def translate(text):
    input_text = "translate English to Thanglish: " + text
    input_ids = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=128).to(model.device)
    output_ids = model.generate(input_ids, max_length=50, num_beams=4)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Try a sample
print(translate("I will call you later."))
print(translate("How are you doing today?"))
print(translate("I am going to the office now."))
print(translate("What is your favorite movie?"))
print(translate("Can we meet at 5 PM today?"))
print(translate("I have a meeting with the team at 3 PM."))
print(translate("We have a conference call scheduled for 2 PM."))
print(translate("Let's brainstorm ideas for the upcoming project."))



call .
?
office-  .
favorite movie?
5 PM- meet  ?
3 PM-  team- meeting .
2 PM-  conference call schedule .
project- brainstorm ideas .


**Model-2**

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
import json
import torch

# Step 1: Load Dataset
with open("/content/en_ta_instruct.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)

data = [{"english": d["input"], "thanglish": d["output"]} for d in raw_data if d.get("input") and d.get("output")]
dataset = Dataset.from_list(data).train_test_split(test_size=0.1)

# Step 2: Load Model & Tokenizer
model_name = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name, legacy=False)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Step 3: Preprocessing
def preprocess(example):
    input_text = "translate English to Thanglish: " + example["english"]
    target_text = example["thanglish"]
    inputs = tokenizer(input_text, max_length=128, padding="max_length", truncation=True)
    targets = tokenizer(target_text, max_length=128, padding="max_length", truncation=True)
    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized_dataset = dataset.map(preprocess, remove_columns=["english", "thanglish"])

# Step 4: Training Configuration
training_args = TrainingArguments(
    output_dir="./t5-thanglish1",
    learning_rate=3e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,                          # Increased epochs for better learning
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs1",
    logging_steps=100,
    evaluation_strategy="epoch",                 # Back in!
    save_strategy="epoch",
    # Removed predict_with_generate
    fp16=torch.cuda.is_available()               # Enable mixed precision if GPU supports it
)

# Step 5: Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
)

# Step 6: Train the Model
trainer.train()

# Step 7: Save the Final Model
model.save_pretrained("t5-thanglish-final1")
tokenizer.save_pretrained("t5-thanglish-final1")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nishakar055 (nishakar055-nonolive) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.085900,0.075311
2,0.066400,0.080154
3,0.066700,0.080091
4,0.068600,0.080056
5,0.000000,nan


('t5-thanglish-final1/tokenizer_config.json',
 't5-thanglish-final1/special_tokens_map.json',
 't5-thanglish-final1/spiece.model',
 't5-thanglish-final1/added_tokens.json')

**Accuracy**

In [ ]:
#!pip install evaluate
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=d0398e6218e48ee6a43b9241e49c7084e9cbaec2b7ed47d285068bbdc1aafc52
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from datasets import Dataset
import evaluate
import json
from tqdm import tqdm

# Load test dataset
with open("en_ta_instruct.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)

# Prepare data
data = [{"english": d["input"], "thanglish": d["output"]} for d in raw_data if d.get("input") and d.get("output")]
dataset = Dataset.from_list(data).train_test_split(test_size=0.1)
test_data = dataset["test"]

# Load model
model = T5ForConditionalGeneration.from_pretrained("t5-thanglish-final1")
tokenizer = T5Tokenizer.from_pretrained("t5-thanglish-final1")
model.eval()

# Load evaluation metrics
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

# Generate predictions
predictions = []
references = []

for example in tqdm(test_data, desc="Generating translations"):
    input_text = "translate English to Thanglish: " + example["english"]
    input_ids = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=128)
    output_ids = model.generate(input_ids, max_length=128, num_beams=4)
    pred = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Check if prediction is empty and handle it
    if not pred.strip():  # If prediction is empty
        print(f"Warning: Empty prediction for input: {example['english']}")
        # Option 1: Assign a default value to avoid zero length
        pred = " "  # Or any placeholder
        # Option 2: Skip this example for BLEU calculation
        # continue

    predictions.append(pred.strip())
    references.append(example["thanglish"].strip())

# Compute metrics
bleu_result = bleu.compute(predictions=predictions, references=[[ref] for ref in references])
rouge_result = rouge.compute(predictions=predictions, references=references)

# Print results
print("\n=== Evaluation Results ===")
print(f"BLEU Score    : {bleu_result['bleu'] * 100:.2f}")
print(f"ROUGE-L Score : {rouge_result['rougeL'] * 100:.2f}")


Generating translations:   0%|          | 1/1598 [00:00<22:55,  1.16it/s]

Generating translations:   0%|          | 2/1598 [00:01<16:05,  1.65it/s]

Generating translations:   0%|          | 3/1598 [00:01<13:29,  1.97it/s]

Generating translations:   0%|          | 4/1598 [00:02<12:30,  2.12it/s]

Generating translations:   0%|          | 5/1598 [00:02<12:10,  2.18it/s]

...

Generating translations: 100%|█████████▉| 1595/1598 [07:50<00:00,  3.23it/s]

Generating translations: 100%|█████████▉| 1596/1598 [07:50<00:00,  3.15it/s]

Generating translations: 100%|█████████▉| 1597/1598 [07:50<00:00,  3.30it/s]

Generating translations: 100%|██████████| 1598/1598 [07:50<00:00,  3.39it/s]

ZeroDivisionError: float division by zero